In [2]:
%load_ext autoreload
%autoreload 2
import feature_extract
import sys
sys.path.append("..")
import torch
import psql_methods as psql
import pickle
import alchemy_methods as alc
from tqdm import tqdm
import numpy as np
from more_itertools import chunked
import pandas as pd
import snob_utils as snob

The goal of this notebook is to build the required sales data for snob effect analysis


In [ ]:
# model = torch.hub.load('facebookresearch/dino:main', 'dino_vits8')
model_string = 'dinov2_vits14'
data_path = '/global/scratch/tlundy/NFT_Research/nft_research/Dino/images'
out_path = f'/global/scratch/tlundy/NFT_Research/nft_research/Dino/features/{model_string}'
# out_path = f'/global/scratch/tlundy/NFT_Research/nft_research/Dino/features/test'


In [ ]:
#We assume the features have already been computed by the feature_extract_notebook
feature_path = out_path+'/testfeat.pth'
features = torch.load(feature_path)
labels = feature_extract.get_labels(data_path)
file_names = feature_extract.get_filenames(data_path)
print(file_names[0])
print(labels)

In [ ]:
commands = ["SELECT * from collectiontoaddress"]
data = psql.execute_commands(commands)
slug_to_contract = {x[0]:x[1] for x in data}

In [ ]:
import pandas as pd
features_list = features.tolist()

# Create a pandas DataFrame
data = {'Label': labels.tolist(), 'Features': features_list,'Collection':[x[0] for x in file_names],
        'NFT_num':[x[1] for x in file_names], 'Contract': [slug_to_contract.get(x[0],None) for x in file_names]}
df = pd.DataFrame(data)


In [ ]:
del data
del features
del features_list


In [ ]:
#Find NFTS for which we already have sales data in case we need to start at an intermediate point
command = "Select distinct contract from nfttosales_2"
rows = psql.execute_commands([command])
rows = [x[0] for x in rows] 

In [ ]:
#Filter out NFT collections for which we already have sales data
selected_columns = df[['Contract', 'NFT_num']].to_numpy().tolist()
print(len(selected_columns))
selected_columns = [x for x in selected_columns if x[0] not in rows]
print(len(selected_columns))
sales_grab = list(chunked(selected_columns,10_000))


In [ ]:

for sale_chunk in sales_grab:
    sales = alc.NFT_to_sales(sale_chunk)
    commands = []
    data_list = []
    command = "INSERT INTO nfttosales_2 (contract, token_id, sale_price) VALUES (%s, %s, %s) returning token_id"
    for sale in sales:
        commands.append(command)
        data_list.append(sale)
    psql.execute_commands(commands,data_list)